# Analyse de sentiment avec SparkML

Données Amazon Reviews:    
L'ensemble de données des avis Amazon se compose d'avis d'Amazon. Les données couvrent une période de 18 ans d'avis jusqu'en mars 2013 et comptent plus de 7 millions d'avis.
   
   
L'objectif de ce TP sera de faire de l'analyse de sentiment en utilisant un classifieur et la librairie Spark.


#1.Installation de Spark

In [ ]:
# Installation de Spark depuis le script d'installation
!git clone https://github.com/bamedro/training-bigdata.git
!chmod u+x ./training-bigdata/4-spark/setup-spark-on-ubuntu.sh
!./training-bigdata/4-spark/setup-spark-on-ubuntu.sh

In [ ]:
# Définition des variables d'environnement (adapter si besoin)
!ls /usr/lib/jvm/ /content
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

#2.Téléchargement des données

1. Créer un compte sur Kaggle (adresse mail)
2. Connectez-vous sur Kaggle, puis accédez à votre profil Kaggle et créez un nouveau Token API Kaggle.
Cela téléchargera automatiquement un fichier **kaggle.json** sur votre PC
3. Importer ce fichier **kaggle.json** dans votre environnement Google Colab en utilisant la fonction **files.upload()** et **shutil.copy**
4. installer la bibliothèque kaggle
5. Télécharger et décompresser le jeu de données à l'aide de la fonction **kaggle.api.dataset_download_files**
6. Renommer les fichiers test.csv et train.csv en TP_sparkML_test.csv et TP_sparkML_train.csv

Jeu de données Amazon Review disponible à l'adresse suivante:
https://www.kaggle.com/datasets/kritanjalijain/amazon-reviews/data

### 2.3 Importer kaggle.json

In [ ]:
from google.colab import files
import shutil

# Utilisez files.upload() pour charger le fichier kaggle.json obtenu lors de votre inscription
uploaded = files.upload()

!mkdir /root/.kaggle
shutil.copy('kaggle.json', '/root/.kaggle/kaggle.json')

### 2.4 Installer et importer Kaggle

In [ ]:
!pip install kaggle

In [ ]:
import kaggle

In [ ]:
# Rendre la clé non lisible par d'autres utilisateurs du système
!chmod 600 /root/.kaggle/kaggle.json

### 2.5 Jeu de données
Utiliser la commande *kaggle* pour :
- Vérifier la disponibilité de votre jeu de données kritanjalijain/amazon-reviews
- Télécharger et décompresser le jeu de données à l'aide de la fonction kaggle.api.dataset_download_files

In [ ]:
# Lister les jeux de données disponibles
...

ref                                        title                        size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------  --------------------------  -----  -------------------  -------------  ---------  ---------------  
kritanjalijain/amazon-reviews              Amazon reviews                1GB  2021-05-15 09:45:40           9491        120  1.0              
kritanjalijain/maestropianomidi            Maestro-Piano-Midi           56MB  2021-04-29 07:31:56            424         14  0.8235294        
kritanjalijain/beethoven-midi              beethoven_midi              347KB  2021-04-29 08:41:42             65          4  0.1875           
kritanjalijain/outfititems                 outfit-items                  6GB  2022-02-13 12:46:33            324         17  0.75             
kritanjalijain/music-midi-dataset          music_midi_dataset           56KB  2021-04-26 15:51:32            236          7  0.8125           

In [ ]:
# Télécharger le jeu de données "kritanjalijain/amazon-reviews"
# et décompressez-le
...


À présent vous devriez avoir dans vos fichiers Google Colab:
- test.csv
- train.csv

Pour plus de clarté nous allons renommer ces fichiers TP_sparkML_test.csv et TP_sparkML_train.csv

In [ ]:
!mv test.csv TP_sparkML_test.csv
!mv train.csv TP_sparkML_train.csv

##3. Aperçu des données

1. Créer une SparkSession appelée *TP_SparkML*
2. Charger les données train et test dans des spark Dataframes
3. Observer les données: répartitions, quantité, intitulé des colonnes...

###3.1 Créer la SparkSession

In [ ]:
# Importez les modules nécessaires
from pyspark.sql import SparkSession

In [ ]:
# Créez une SparkSession
spark = ...

###3.2 Chargement des données CSV dans des Spark Dataframes
Créer deux Dataframes, *test* et *train*, à partir des fichiers CSV correspondants

In [ ]:
# Chemins
train_path = '/content/TP_sparkML_train.csv'
test_path = '/content/TP_sparkML_test.csv'
# spark Dataframes
train = ...
test = ...

In [ ]:
train.show(5)

+---+--------------------+--------------------+
|_c0|                 _c1|                 _c2|
+---+--------------------+--------------------+
|  2|Stuning even for ...|This sound track ...|
|  2|The best soundtra...|I'm reading a lot...|
|  2|            Amazing!|"This soundtrack ...|
|  2|Excellent Soundtrack|I truly like this...|
|  2|Remember, Pull Yo...|If you've played ...|
+---+--------------------+--------------------+
only showing top 5 rows



### 3.3 Étudier la taille des dataframes et la répartition des classes

In [ ]:
# Afficher le nombre de lignes dans le trainset, puis dans le testset
print("Nombre de lignes du jeu de données train:", ...)
print("Nombre de lignes du jeu de données test:", ...)

Nombre de lignes du jeu de données train: 3600000
Nombre de lignes du jeu de données test: 400000


In [ ]:
# Récupérer le schéma du Dataframe
train.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [ ]:
# Afficher la répartition des classes positives et négatives dans train
...
class_counts_train.show()

+---+-------+
|_c0|  count|
+---+-------+
|  1|1800000|
|  2|1800000|
+---+-------+



In [ ]:
# Afficher la répartition des classes positives et négatives dans test
...
class_counts_test.show()

+---+------+
|_c0| count|
+---+------+
|  1|200000|
|  2|200000|
+---+------+



### Que constatez-vous sur la répartition de votre jeu de données?

...

### Est-ce que chaque review contient un commentaire?

In [ ]:
# Appliquer un filtre sur train pour ne conserver que les commentaires vides
...

# Affichez les lignes de train où la colonne _c2 est vide
empty_reviews_train.show()

+---+---+---+
|_c0|_c1|_c2|
+---+---+---+
+---+---+---+



In [ ]:
# Appliquer un filtre sur test pour ne conserver que les commentaires vides
...

# Affichez les lignes de test où la colonne _c2 est vide
empty_reviews_test.show()

+---+---+---+
|_c0|_c1|_c2|
+---+---+---+
+---+---+---+



### Quelles seront les étapes de préparation des données?

Détailler les différentes étapes qui permettront d'utiliser ces données pour entraîner un modèle.

1. **Suppression du bruit** : ....

2. **Tokenisation** : ...

3. **Suppression des stopwords** : ...

4. **Représentation du texte** : ...

#4. Préparation des données

1. Suppression du bruit: supprimer les informations non pertinentes (les caractères spéciaux, les chiffres, les signes de ponctuation et les liens URL)
2. Tokenisation du texte
3. Suppression des Stopwords
4. Représenter le texte dans un format adapté au ML (TF/IDF)
5. Créer une colonne label qui convertit les classes en 0 ou 1 et de type integer
6. Appliquer sur test

Vous pourrez vous aider des exemples de code disponibles dans l'archive Spark téléchargée au début du TP :
/content/spark-3.5.0-bin-hadoop3/examples/src/main/python/ml

In [ ]:
import re
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, HashingTF, IDF

### 4.1 Suppression du bruit avec UDF (User Defined Function)

Les UDF permettent d'appliquer des fonctions sur les lignes d'un Dataframe.

Créer une fonction de suppression des caractères spéciaux et l'appliquer
avec UDF

In [ ]:
# Créez une fonction UDF pour supprimer les caractères spéciaux,
# les chiffres et les URL. Cette fonction passe également
# la chaine en minuscule
def clean_text(text):
  ...
  return ...

In [ ]:
# Appliquez la fonction UDF à votre colonne de reviews (_c2)
...

In [ ]:
train.show(5)

+---+--------------------+--------------------+
|_c0|                 _c1|                 _c2|
+---+--------------------+--------------------+
|  2|Stuning even for ...|this sound track ...|
|  2|The best soundtra...|im reading a lot ...|
|  2|            Amazing!|this soundtrack i...|
|  2|Excellent Soundtrack|i truly like this...|
|  2|Remember, Pull Yo...|if youve played t...|
+---+--------------------+--------------------+
only showing top 5 rows



### 4.2 Tokenisation du texte

In [ ]:
# Tokenisation du texte
...
train.show(5)

+---+--------------------+--------------------+--------------------+
|_c0|                 _c1|                 _c2|               words|
+---+--------------------+--------------------+--------------------+
|  2|Stuning even for ...|this sound track ...|[this, sound, tra...|
|  2|The best soundtra...|im reading a lot ...|[im, reading, a, ...|
|  2|            Amazing!|this soundtrack i...|[this, soundtrack...|
|  2|Excellent Soundtrack|i truly like this...|[i, truly, like, ...|
|  2|Remember, Pull Yo...|if youve played t...|[if, youve, playe...|
+---+--------------------+--------------------+--------------------+
only showing top 5 rows



### 4.3 Suppression des Stopwords

In [ ]:
# Suppression des stopwords
...
train.show(5)

+---+--------------------+--------------------+--------------------+--------------------+
|_c0|                 _c1|                 _c2|               words|         clean_words|
+---+--------------------+--------------------+--------------------+--------------------+
|  2|Stuning even for ...|this sound track ...|[this, sound, tra...|[sound, track, be...|
|  2|The best soundtra...|im reading a lot ...|[im, reading, a, ...|[im, reading, lot...|
|  2|            Amazing!|this soundtrack i...|[this, soundtrack...|[soundtrack, favo...|
|  2|Excellent Soundtrack|i truly like this...|[i, truly, like, ...|[truly, like, sou...|
|  2|Remember, Pull Yo...|if youve played t...|[if, youve, playe...|[youve, played, g...|
+---+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



### 4.4 Vectorisation des textes

In [ ]:
# TF (Term Frequency)
hashingTF = ...
train = hashingTF.transform(train)

# IDF (Inverse Document Frequency)
idf = ...
train = idf.transform(train)
train.show(5)

+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0|                 _c1|                 _c2|               words|         clean_words|       text_features|            features|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  2|Stuning even for ...|this sound track ...|[this, sound, tra...|[sound, track, be...|(1000,[28,69,92,1...|(1000,[28,69,92,1...|
|  2|The best soundtra...|im reading a lot ...|[im, reading, a, ...|[im, reading, lot...|(1000,[1,23,38,60...|(1000,[1,23,38,60...|
|  2|            Amazing!|this soundtrack i...|[this, soundtrack...|[soundtrack, favo...|(1000,[0,29,57,86...|(1000,[0,29,57,86...|
|  2|Excellent Soundtrack|i truly like this...|[i, truly, like, ...|[truly, like, sou...|(1000,[33,45,57,6...|(1000,[33,45,57,6...|
|  2|Remember, Pull Yo...|if youve played t...|[if, youve, playe...|[youve, 

In [ ]:
# Visionner la première ligne de votre train pour observer toutes les étapes du traitement des données
train.show(n=1, truncate=False)

+---+------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------

### 4.5 Convertir et nommer la colonne des labels

In [ ]:
from pyspark.sql.functions import col

In [ ]:
# Convertir la colonne _c0 en type integer et créer une nouvelle colonne "label"
# Convertir les labels pour qu'ils aient la valeur 0 ou 1 (binaire)
# à la place de 1 ou 2 pour le modèle
...


In [ ]:
train.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- clean_words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- text_features: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: integer (nullable = true)



In [ ]:
# Sélectionnez les colonnes 'label' et 'features' dans le DataFrame train
df_train = ...

df_train.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(1000,[28,69,92,1...|    1|
|(1000,[1,23,38,60...|    1|
|(1000,[0,29,57,86...|    1|
|(1000,[33,45,57,6...|    1|
|(1000,[35,43,45,6...|    1|
+--------------------+-----+
only showing top 5 rows



### 4.6 Préparation du df_test

In [ ]:
# Préparer le jeu de données de test avec la même procédure

# Appliquez la fonction UDF à vos colonnes
test = ...
# Tokenisation du texte
test = ...
# Suppression des stopwords
test = ...
# TF (Term Frequency)
test = ...
# IDF (Inverse Document Frequency)
test = ...
# Labels
test = ...
# Sélectionnez les colonnes 'label' et 'features' dans le DataFrame train
df_test = ...

df_test.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(1000,[1,48,83,11...|    1|
|(1000,[23,40,48,8...|    1|
|(1000,[34,56,86,1...|    0|
|(1000,[34,56,110,...|    1|
|(1000,[51,55,56,5...|    1|
+--------------------+-----+
only showing top 5 rows



# 5. Modélisation
1. Importez les classes nécessaires de Spark MLlib pour le modèle et les évaluations de modèle
2. Créez et entraînez un modèle
3. Utilisez le modèle entraîné pour faire des prédictions sur les données de test
4. Évaluez la performance du modèle en utilisant une métrique appropriée, comme la précision (accuracy)

### 5.1 Importations

Choisir un type de modèle parmi les propositions

In [ ]:
# from pyspark.ml.classification import LinearSVC
# from pyspark.ml.classification import LogisticRegression
# from pyspark.ml.classification import NaiveBayes
# from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
# Importer un Evaluater pour évaluer l'efficacité de votre modèle
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

### 5.2 Créer un modèle de votre choix

In [ ]:
svm_model = ...

### 5.3 Prédictions des données test

In [ ]:
predictions = ...

### 5.4 Évaluer la performance

In [ ]:
evaluator = ...

print(f"Accuracy: {accuracy}")

Accuracy: 0.7618025
